In [ ]:
try:
    # settings colab:
    import google.colab
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Análisis de Componentes Principales (PCA)

---

<a id="section_toc"></a> 

## Tabla de Contenidos

[Intro](#section_intro)

[PCA para descubrir variables latentes](#section_latentes)

[PCA para reducir ruido](#section_ruido)






<a id="section_intro"></a> 
## Intro

[volver a TOC](#section_toc)
    
El análisis de componentes principales (PCA) es una de las técnicas más famosas para la reducción de dimensiones, extracción de características y visualización de datos. 

PCA se define por la transformación de un espacio vectorial de alta dimensión en un espacio de baja dimensión. 

Consideremos la visualización de datos de 10 dimensiones, no es posible mostrar efectivamente la forma de una distribución de datos de una dimensionalidad tan alta. PCA proporciona una forma eficiente de reducir la dimensionalidad (es decir, de 10 a 2) permitiendo visualizar la forma de la distribución de datos. 

PCA también es útil en modelos de clasificación robustos donde se proporciona un número considerablemente pequeño de datos de entrenamiento de alta dimensión. Al reducir las dimensiones de los conjuntos de datos de aprendizaje, PCA proporciona un método eficaz y eficiente para la descripción y clasificación de datos.
    
En resumen, PCA tiene una variedad de aplicaciones:
1. Reducción de la dimensionalidad
2. Visualización
3. Eliminación el ruido
4. Generación nuevos features en el dataset

**Feature extraction**

Supongamos que tenemos diez variables independientes. En la extracción de características, creamos diez variables independientes "nuevas", donde cada variable independiente "nueva" es una combinación de cada una de las diez variables independientes "antiguas". 
Creamos estas nuevas variables independientes de una manera específica y ordenamos estas nuevas variables según cuán bien representan los datos originales.

¿Dónde entra en juego la reducción de dimensionalidad? 

Mantenemos tantas variables independientes nuevas como queramos, pero eliminamos las "menos importantes". 

Debido a que ordenamos las nuevas variables según cuán bien representan los datos originales, sabemos qué variable es la más importante y la menos importante. Pero, y aquí está el truco, debido a que estas nuevas variables independientes son combinaciones de las anteriores, aún conservamos las partes más valiosas de nuestras antiguas variables, ¡incluso cuando eliminamos una o más de estas "nuevas" variables!

El análisis de componentes principales es una técnica para la extracción de características, por lo que combina nuestras variables de entrada de una manera específica, ¡entonces podemos descartar las variables "menos importantes" mientras conservamos las partes más valiosas de todas las variables! 

Como beneficio adicional, cada una de las "nuevas" variables después de PCA son **independientes** entre sí. Esto es un beneficio porque los supuestos de un modelo lineal requieren que nuestras variables explicativas sean independientes entre sí. Si decidimos ajustar un modelo de regresión lineal con estas "nuevas" variables, este supuesto necesariamente se cumplirá.

**¿Cuándo debemos usar PCA?**

* ¿Deseamos reducir el número de variables, pero no podemos identificar qué variables no son explicativas?

* ¿Deseamos asegurar que las variables sean independientes entre sí?

* ¿Estamos cómodos haciendo que las variables independientes sean menos interpretables?

Si la respuesta es "sí" a las tres preguntas, entonces PCA es un buen método. Si la respuesta a la pregunta 3 es "no", no debemos usar PCA.

## Loadings y scores

PCA se basa en la descomposición de una matriz de features X en dos matrices V y U:

<img src='img/hl_pca_matmult.png' align='left'/>

Las dos matrices V y U son ortogonales. 

La matiz V se llama **loadings**, y la matriz U se llama **scores**. 

Los loadings pueden pensarse como el peso de cada variable original para cada componente principal. 

La matriz U tiene los datos originales representados en el sistema de coordenadas definido por las componentes principales (un sistema de coordenadas rotado).

## Imports

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits

<a id="section_latentes"></a> 
## PCA para descubrir variables latentes

[volver a TOC](#section_toc)


Vamos ver un ejemplo presentado en *An Introduction to Statistical Learning - with Applications in R* de Gareth James, Daniela Witten, Trevor Hastie y Robert Tibshirani.

En este ejemplo vamos a ver datos de arrestos en ciudades de USA.

In [ ]:
df = pd.read_csv('../Data/USArrests.csv', index_col=0)
df.info()

In [ ]:
df.head()

Observamos la media y varianza de las variables:

In [ ]:
print("Media de las variables: ")
print(df.mean(axis=0))

print('\n')

print("Varianza de las variables: ")
print(df.var(axis=0))

Normalizamos los datos utilizando StandardScaler:

In [ ]:
std_sclr = StandardScaler()
df_std = pd.DataFrame(std_sclr.fit_transform(df), index=df.index, columns=df.columns)

Observamos nuevamente la media y varianza de las variables:

In [ ]:
print("Media de las variables: ")
print(df_std.mean(axis=0))

print('\n')

# Observamos nuevamente la varianza de las variables:
print("Varianza de las variables: ")
print(df_std.var(axis=0))

Instanciamos un objeto de la clase PCA. Al no especificar en el argumento el número de componentes, PCA va a conservar todos los CP.

Calculamos las componentes principales con el método `fit` sobre los datos normalizados


In [ ]:
pca_arrests = PCA()
pca_arrests.fit(df_std)

Creamos un DataFrame con los loadings.

Recordemos que los loadings representan el peso de cada variable original para cada componente principal

En este caso podemos ver que:

* la variable Assault es la que más contribuye a la pimera componenete principal

* la variable UrbanPop es la  que más contribuye a la segunda componenete principal

Miramos el valor absoluto (módulo) del loading para decidir cuál es el que más contribuye.

In [ ]:
pca_loadings = pd.DataFrame(pca_arrests.components_.T, index=df.columns, columns=['PC1', 'PC2', 'PC3', 'PC4'])
pca_loadings

Recordemos que un 

* conjunto de vectores es ortonormal, si es un conjunto ortogonal y la norma de cada uno de sus vectores es igual a 1.

* dos vectores son ortogonales si su producto interno da 0.

Verificamos la ortonormalidad de los componentes principales:

In [ ]:
pca_loadings.T.dot(pca_loadings)

Escribimos los datos originales normalizados en el sistema de coordenadas del espacio generado por las componenentes principales

In [ ]:
df_pca = pd.DataFrame(pca_arrests.fit_transform(df_std), columns=['PC1', 'PC2', 'PC3', 'PC4'],\
                      index=df_std.index)
df_pca.head()

Calculamos la media y varianza de estas variables

In [ ]:
print("Media de los CP: ")
print(df_pca.mean(axis=0))

print('\n')

# Observamos la varianza de las variables:
print("Varianza de los CPs: ")
print(df_pca.var(axis=0))

Vamos a usar como features de nuestro conjunto de datos sólo dos variables: PC1 y PC2.

Y representamos cada registro del dataset original usando como coordena x el valor de PC1 y como coordenada y el valor de PC2
        

In [ ]:
fig , ax1 = plt.subplots(figsize=(9,7))

ax1.set_xlim(-3.5,3.5)
ax1.set_ylim(-3.5,3.5)

# Ploteamos a los Estados en el espacio de los Componentes Principales 1 y 2
for i in df_pca.index:
    ax1.annotate(i, (df_pca.PC1.loc[i], -df_pca.PC2.loc[i]), ha='center')

# Ploteamos las líneas de referencia
ax1.hlines(0,-3.5,3.5, linestyles='dotted', colors='grey')
ax1.vlines(0,-3.5,3.5, linestyles='dotted', colors='grey')

ax1.set_xlabel('Primer Componente Principal')
ax1.set_ylabel('Segundo Componente Principal')
    
# Creamos ejes secundarios
ax2 = ax1.twinx().twiny() 

ax2.set_ylim(-1,1)
ax2.set_xlim(-1,1)
ax2.tick_params(axis='y', colors='orange')
ax2.set_xlabel('Vectores de loadings de los Componentes Principales', color='orange')

# Ploteamos a las variables originales en relación a los Componentes Principales 1 y 2
for i in pca_loadings[['PC1', 'PC2']].index:
    ax2.annotate(i, (pca_loadings.PC1.loc[i], -pca_loadings.PC2.loc[i]), color='orange')

# Plot vectors
ax2.arrow(0,0,pca_loadings.PC1[0], -pca_loadings.PC2[0])
ax2.arrow(0,0,pca_loadings.PC1[1], -pca_loadings.PC2[1])
ax2.arrow(0,0,pca_loadings.PC1[2], -pca_loadings.PC2[2])
ax2.arrow(0,0,pca_loadings.PC1[3], -pca_loadings.PC2[3]);

De esta forma podemos ver gráficamente la relación entre las variables originales y los primeros 2 componentes principales.


Veamos la varianza explicada de cada componente principal:

In [ ]:
pca_arrests.explained_variance_

Veamos el ratio la varianza explicada por cada componente principal

Vemos que

* la primera componente principal explica el 62% de la varianza de los datos

* la segunda componente principal explica el 25% de la varianza de los datos

* la tercera componente principal explica el 9% de la varianza de los datos

* la cuarta componente principal explica el 4% de la varianza de los datos


In [ ]:
pca_arrests.explained_variance_ratio_

Graficamos el porcentaje de varianza explicada en función de la cantidad de componentes

In [ ]:
plt.figure(figsize=(7,5))

plt.plot([1,2,3,4], pca_arrests.explained_variance_ratio_, '-o', label='Componente individual')
plt.plot([1,2,3,4], np.cumsum(pca_arrests.explained_variance_ratio_), '-s', label='Acumulado')

plt.ylabel('Porcentaje de Varianza Explicada')
plt.xlabel('Componentes Principales')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks([1,2,3,4])
plt.legend(loc=2);

En este gráfico podemos observar que usando sólo dos features (PC1 y PC2) explicamos más del 80% de la varianza del conjunto original

<a id="section_ruido"></a> 
## PCA para reducir ruido

[volver a TOC](#section_toc)


PCA puede ser usado también como "filtro de ruido". La intuición es la siguiente: cada componente con varianza mucho más grande que el ruido debería verse relativamente poco o nada afectado por dicho ruido. Entonces, si reconstruimos los datos usando solamente los componentes de mayor varianza, debería ser posible conservar la mayor parte de la señal y descartar la mayor parte del ruido.

Vamos a usar en este ejemplo el dataset digits que provee `sklearn.datasets`

El dataset de dígitos consiste en imagenes de 8x8 píxeles de dígitos del 0 al 9 escritos a mano.

In [ ]:
digits = load_digits()
digits.data.shape

Definimos una función para plotear los dígitos

In [ ]:
def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    
    for i, ax in enumerate(axes.flat):
        
        ax.imshow(data[i].reshape(8, 8),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))

Ejecutamos la función con los datos de dígitos

In [ ]:
plot_digits(digits.data)

Vamos a introducir ruido en esto datos sumando errores aleatorios que se distribuyen normal con media 0 y desvío 4.

Y veamos cómo quedan los datos con ruido.

In [ ]:
np.random.seed(42)
noisy = np.random.normal(digits.data, 4)
plot_digits(noisy)

Vamos instanciar un objeto PCA que explique el 50% de la varianza.

La cantidad de componenetes necesaria para explicar el 50% de la varianza resultó ser 5, mientras que el dataset original tiene 64 features

In [ ]:
pca_digits = PCA(0.50)
pca = pca_digits.fit(noisy)
pca.n_components_

Calculemos los componentes del dataset con ruido

In [ ]:
components = pca.transform(noisy)
components.shape

Apliquemos ahora la transformación inversa, que a partir de los componentes reconstruya los valores de las x originales.

In [ ]:
filtered = pca.inverse_transform(components)
plot_digits(filtered)

Aunque el dataset resultado tiene menos definición que el original, vemos que el ruido fue eliminado.

Observemos también que en esta visualización tenemos imágenes representadas con 12 features en lugar de las 64 que definian el datatset original. Es decir, descartamos 52 componentes de nuestro dataset.

## PCA para visualizar datos

Vamos a proyectar los datos del dataset de digitos en las primeras dos componenetes principales, y vamos a usar esta proyección para visualizar los datos


In [ ]:
pca_digits_vis = PCA(n_components=2)
projected = pca_digits_vis.fit_transform(digits.data)
print(digits.data.shape)
print(projected.shape)

Definimos una función para plotear los dígitos en 2 dimensiones generados por PCA

In [ ]:
def plot_digits_pca(projection, numbers):
    
    colors = ["#476A2A", "#7851B8", "#BD3430", "#4A2D4E", "#875525",
          "#A83683", "#4E655E", "#853541", "#3A3120", "#535D8E"]
    plt.figure(figsize=(10,10))
    plt.xlim(projection[:,0].min(), projection[:,0].max())
    plt.ylim(projection[:,1].min(), projection[:,1].max())

    for i in range(len(projection)):
        plt.text(projection[i,0], projection[i,1], str(numbers[i]),
                color=colors[numbers[i]], fontdict={'weight':'bold', 'size':9})
        plt.xlabel('Primer Componente Principal')
        plt.ylabel('Segundo Componente Principal')

Ploteamos los dígitos proyectados con PCA

In [ ]:
plot_digits_pca(projected, digits.target)

Vemos que PCA logró encontrar alguna estructura en los datos pero no llegó a separar bien a la mayoría de los números. Por ejemplo, el segundo componente parece distinguir bien entre 0 y 1. 

Un punto importante a remarcar es que estamos ploteando usando las etiquetas de los registros. Es decir, tenemos más información que la que tendríamos en un problema típico de aprendizaje no supervisado. 

Veamos cuál sería el resultado del ploteo sin las etiquetas:

In [ ]:
plt.figure(figsize=(10,10))
plt.xlim(projected[:,0].min(), projected[:,0].max())
plt.ylim(projected[:,1].min(), projected[:,1].max())

for i in range(len(projected)):
    plt.scatter(projected[i,0], projected[i,1], color='b', s=10)

Podemos intuir algún tipo de estructura o separación entre grupos, pero evidentemente los datos son demasiado complejos y no lineales para que PCA pueda capturar correctamente la estructura y separarlos en grupos distintos (acorde a su etiqueta).

## Referencias

* User guide https://scikit-learn.org/stable/modules/decomposition.html#pca

* Documentación https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

* Principal Component Analysis Explained Visually https://setosa.io/ev/principal-component-analysis/

* http://www.statistics4u.info/fundstat_eng/cc_pca_loadscore.html

* Python Data Science Handbook, Jake VanderPlas https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html